## Notebook: Calibration of the Heston Model  

This notebook focuses on pricing a **1-year call option** using the **Heston model** with the following parameters:  

- $v_0 = \sigma^2 = 0.05$  
- $\kappa = 0.5$  
- $\eta = 0.05$  
- $\theta = 0.2$  
- $\rho = -0.75$  
- $S_0 = 100$  
- $q = 0.01$ (dividend yield)  
- $r = 0.05$ (risk-free rate)  

### **Strike Prices**  
We consider three different strike prices:  
$$K = 90, 100, 110$$  

### **Pricing Methods**  
The option price is computed using the following methods:  
1. **Fast Fourier Transform (FFT)** with Simpson’s rule *(see Example Session 2)*.  
2. **Monte Carlo simulation** using the **Euler scheme**.  
3. **Monte Carlo simulation** using the **Milstein scheme**.  


In [ ]:
# imports 
from  tiny_pricing_utils import *
import numpy as np

In [3]:
# Market data
S0 = 100
q = 0.01
r = 0.05
sigma = 0.25
K = [90, 100, 110]
T = 1

In [4]:
# Hetson variables !!eta 
V0 = 0.05
kappa = 0.5 
eta = 0.05
theta = 0.2
rho = -0.75

In [5]:
# variables for FFT pricing (eta != eta_cm)
eta_cm = 0.25
N = 4096
alpha = 1.5
lambda_ = (2 * np.pi) / (N* eta_cm)
b = (N * lambda_) / 2
print(lambda_)
print(b)

0.006135923151542565
12.566370614359172


In [6]:
# Create an instance of the Heston model with the provided data
heston = HestonModel(S0=S0, r=r, q=q, V0=V0, kappa=kappa, eta=eta, theta=theta, rho=rho, T=T, 
                         alpha=alpha, N=N, eta_cm=eta_cm, b=b, strikes=K)

# Simpson's Rule coefficients for pricing
simpson_1 = 1 / 3  # First coefficient
simpson_weights = (3 + (-1) ** np.arange(2, N + 1)) / 3  # Alternating coefficients starting from index 2
simpson_weights = np.concatenate(([simpson_1], simpson_weights))  # Combine with the first coefficient

# Price options using Simpson's rule
option_prices_simpson = heston.price_options(rule="simpson", simpson_weights=simpson_weights)

# Print results for visual confirmation
print(f"Interpolated Option Prices using Simpson's Rule: {option_prices_simpson}")

Interpolated Option Prices using Simpson's Rule: [16.93747985 10.6163149   5.86707143]


In [ ]:
#Monte Carlo pricing with Euler scheme
n = 252
m = 10000
np.random.seed(0)


In [ ]:
def simulate_stock_paths(S0, r, q, sigma,T, N, M):
    dt = 1/365  # Time step size
    stock_paths = np.zeros((M, N + 1))  # Store stock prices for all paths
    stock_paths[:, 0] = S0  # Initial stock price
    
    # Generate random numbers for Brownian motion
    Z = np.random.randn(M, N)
    
    for i in range(1, N + 1):
        stock_paths[:, i] = stock_paths[:, i-1] * np.exp((r - q - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, i-1])
    
    return stock_paths